In [13]:
import pandas as pd

In [3]:
dt_sorted = pd.read_csv("dt_sorted").reset_index()
rf_sorted = pd.read_csv("rf_sorted").reset_index()
xgb_sorted = pd.read_csv("xgb_sorted").reset_index()

In [4]:
dt_sorted.rename(columns={"index": "dt_rank", "features_weights": "dt_weights"}, inplace=True)
rf_sorted.rename(columns={"index": "rf_rank", "features_weights": "rf_weights"}, inplace=True)
xgb_sorted.rename(columns={"index": "xgb_rank", "features_weights": "xgb_weights"}, inplace=True)

In [8]:
df = dt_sorted.merge(rf_sorted, how= 'left',on='features')
df = df.merge(xgb_sorted, how= 'left',on='features')
# df.rename(columns={"features_weights_x": "dt_weights", "features_weights_y": "rf_weights", "features_weights": "xgb_weights"}, inplace=True)

In [11]:
df['final_rank'] = df['dt_rank'] + df['rf_rank'] + df['xgb_rank']
df['weighted_final_rank'] = (df['dt_rank'] + df['rf_rank'] +  df['xgb_rank']) / (df['dt_weights'] + df['rf_weights'] + df['xgb_weights'])

In [12]:
df

,dt_rank,features,dt_weights,rf_rank,rf_weights,xgb_rank,xgb_weights,final_rank,weighted_final_rank
0,0,num_bids_per_device,0.541402,0,0.097697,0,0.215621,0,0.000000e+00
1,1,percentage_of_auctions_above_threshold,0.058821,2,0.058382,3,0.050770,6,3.571999e+01
2,2,time_to_bid,0.052277,6,0.046982,12,0.020832,20,1.665408e+02
3,3,num_bids_per_ip,0.038356,10,0.033952,15,0.017108,28,3.131447e+02
4,4,mobile,0.030118,17,0.016915,22,0.013663,43,7.084525e+02
5,5,on_url_that_has_a_bot_mean,0.029920,13,0.021911,10,0.024928,28,3.647774e+02
6,6,max_devices_per_auction,0.026281,24,0.013788,8,0.029041,38,5.498513e+02
7,7,num_last_bids,0.024571,26,0.013640,9,0.027284,42,6.412675e+02
8,8,sporting goods,0.022506,37,0.008272,29,0.010999,74,1.771280e+03
9,9,std_url_per_auction,0.019009,35,0.008682,30,0.010762,74,1.924441e+03


In [21]:
final_rank_df_10 = df.sort_values(by=['final_rank']).head(10)
final_rank_df_15 = df.sort_values(by=['final_rank']).head(15)
final_rank_df_20 = df.sort_values(by=['final_rank']).head(20)
final_rank_df_25 = df.sort_values(by=['final_rank']).head(25)
final_rank_df_30 = df.sort_values(by=['final_rank']).head(30)
final_rank_df_35 = df.sort_values(by=['final_rank']).head(35)
final_rank_df_40 = df.sort_values(by=['final_rank']).head(40)
# final_rank_df
final_rank_list_10 = final_rank_df_10['features'].to_list()
final_rank_list_15 = final_rank_df_15['features'].to_list()
final_rank_list_20 = final_rank_df_20['features'].to_list()
final_rank_list_25 = final_rank_df_25['features'].to_list()
final_rank_list_30 = final_rank_df_30['features'].to_list()
final_rank_list_35 = final_rank_df_35['features'].to_list()
final_rank_list_40 = final_rank_df_40['features'].to_list()

print("10 Features")
print(final_rank_list_10)
print("")
print("15 Features")
print(final_rank_list_15)
print("")
print("20 Features")
print(final_rank_list_20)
print("")
print("25 Features")
print(final_rank_list_25)
print("")
print("30 Features")
print(final_rank_list_30)
print("")
print("35 Features")
print(final_rank_list_35)
print("")
print("40 Features")
print(final_rank_list_40)

10 Features
['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile']

15 Features
['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry']

20 Features
['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile', 'no_of_auction_exceeds_threshold', 'num_first_bids', 'ip', 'url', 'jewelry', 'num_bids_per_auction', 'max_ip_per_auction', 'inst_resp', 'computers', 'url_entropy']

25 Features
['num_bids_per_device', 'percentage_of_auctions_abo

In [12]:
final_weighted_rank_df = df.sort_values(by=['weighted_final_rank']).head(21)
# final_weighted_rank_df
final_weighted_rank_df['features'].to_list()

['num_bids_per_device',
 'percentage_of_auctions_above_threshold',
 'time_to_bid',
 'num_bids_per_country',
 'num_bids_per_ip',
 'on_url_that_has_a_bot_mean',
 'no_of_auction_exceeds_threshold',
 'max_devices_per_auction',
 'auction',
 'num_last_bids',
 'inst_resp',
 'num_bids_per_auction',
 'mobile',
 'computers',
 'num_first_bids',
 'jewelry',
 'ip',
 'url',
 'num_bids',
 'perc_inst_resp',
 'max_ip_per_auction']

In [14]:
dt_new_sorted = pd.read_csv("../Experiments/dt_features.csv").reset_index()
rf_new_sorted = pd.read_csv("../Experiments/rf_features.csv").reset_index()
xgb_new_sorted = pd.read_csv("../Experiments/xgb_features.csv").reset_index()

In [16]:
dt_new_sorted.rename(columns={"index": "dt_rank", "features_weights": "dt_weights"}, inplace=True)
rf_new_sorted.rename(columns={"index": "rf_rank", "febatures_weights": "rf_weights"}, inplace=True)
xgb_new_sorted.rename(columns={"index": "xgb_rank", "features_weights": "xgb_weights"}, inplace=True)

In [28]:
df_new = dt_new_sorted.merge(rf_new_sorted, how= 'left',on='Features')
df_new = df_new.merge(xgb_new_sorted, how= 'left',on='Features')

In [32]:
df_new['final_rank'] = df_new['dt_rank'] + df_new['rf_rank'] + df_new['xgb_rank']
final_rank_df_10_new = df_new.sort_values(by=['final_rank']).head(10)
final_rank_list_10_new = final_rank_df_10_new['Features'].to_list()
print(final_rank_list_10_new)

['num_bids', 'num_bids_per_device', 'inst_resp', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 'std_country_per_auction', 'time_to_bid', 'max_devices_per_auction', 'max_ip_per_auction', 'auction']


In [33]:
'num_bids', 'num_bids_per_device', 'inst_resp', 'num_bids_per_ip', 'on_url_that_has_a_bot_mean', 
'std_country_per_auction', 'time_to_bid', 'max_devices_per_auction', 'max_ip_per_auction', 'auction'

['num_bids_per_device', 'percentage_of_auctions_above_threshold', 'time_to_bid', 'num_bids_per_ip', 
 'on_url_that_has_a_bot_mean', 'num_bids_per_country', 'auction', 'max_devices_per_auction', 'num_last_bids', 'mobile']

('std_country_per_auction',
 'time_to_bid',
 'max_devices_per_auction',
 'max_ip_per_auction',
 'auction')

In [ ]:
# # trying selectkbest
# selector = SelectKBest(mutual_info_regression, k = 20)
# selector.fit(X,y)
# selectkbest_support = selector.get_support()

In [ ]:
# # trying rfe
# rfe_random_forest = RFE(estimator=RandomForestClassifier(), n_features_to_select = 20, step = 1)
# rfe_random_forest.fit(X,y)
# rfe_random_forest_support = rfe_random_forest.get_support()

# rfe_xgb = RFE(estimator=XGBClassifier(), n_features_to_select = 20, step = 1)
# rfe_xgb.fit(X,y)
# rfe_xgb_support = rfe_xgb.get_support()

# rfe_gb = RFE(estimator=GradientBoostingClassifier(), n_features_to_select = 20, step = 1)
# rfe_gb.fit(X,y)
# rfe_gb_support = rfe_gb.get_support()

# rfe_logistic = RFE(estimator=LogisticRegression(), n_features_to_select = 20, step = 1)
# rfe_logistic.fit(X,y)
# rfe_logistic_support = rfe_logistic.get_support()

In [ ]:
# # trying selectfrommodel
# sfm_random_forest = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=20)
# sfm_random_forest.fit(X,y)
# sfm_random_forest_support = sfm_random_forest.get_support()

# sfm_xgb = SelectFromModel(XGBClassifier(), max_features=20)
# sfm_xgb.fit(X,y)
# sfm_xgb_support = sfm_xgb.get_support()

# sfm_gb = SelectFromModel(GradientBoostingClassifier(n_estimators=100), max_features=20)
# sfm_gb.fit(X,y)
# sfm_gb_support = sfm_gb.get_support()

# # sfm_logistic = SelectFromModel(LogisticRegression(), max_features=20)
# # sfm_logistic.fit(X,y)
# # sfm_logistic_support = sfm_logistic.get_support()

In [ ]:
# # https://towardsdatascience.com/the-5-feature-selection-algorithms-every-data-scientist-need-to-know-3a6b566efd2
# feature_selection_df = pd.DataFrame({'Feature':X.columns, 'SelectKBest':selectkbest_support, 
#                                      'RFE Random Forest':rfe_random_forest_support, 
#                                      'RFE XGB':rfe_xgb_support, 'RFE GB':rfe_gb_support,
#                                      'RFE Logistic Regression':rfe_logistic_support, 
#                                      'SFM Random Forest':sfm_random_forest_support, 'SFM XGB':sfm_xgb_support,
#                                      'SFM GB':sfm_gb_support})
# # count the selected times for each feature
# feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# # display the top 100
# feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
# feature_selection_df.index = range(1, len(feature_selection_df)+1)
# # feature_selection_df.head(num_feats)
# feature_selection_df